# In the name of God

In this notebook, the predicted values of three main time series — photovoltaic (PV) generation, wind power generation, and load demand of the microgrid — are used in the grid_connected mode of the microgrid energy management system to simulate the system’s performance over the entire one-year operational period.

In [1]:
sequence_length = 24
forecast_horizon = 24
epsilon = 0.5
ratio = 1.0


In [2]:
# Adding src to path:
import sys
import os

sys.path.insert(0, os.path.abspath('../../src'))

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyomo.environ as pyo
from tqdm import tqdm
import datetime
from components_v2 import Load, Generator, Storage, Renewable,  ExGrid, MicroGrid
from Optimization_function_rounded import  timeseries_to_dictionary, create_time_window_dicts_modified, round_value, milp_opti_model_connected
import json

In [ ]:
# Importing dataset:
data = pd.read_csv(r"../../data/dataset.csv",
                   parse_dates=['time'],
                   index_col=['time'])
data

,pv_production,wind_production,consumption,spot_market_price,precip_1h:mm,precip_type:idx,prob_precip_1h:p,clear_sky_rad:W,clear_sky_energy_1h:J,diffuse_rad:W,...,t_50m:C,relative_humidity_50m:p,dew_point_50m:C,wind_speed_50m:ms,wind_dir_50m:d,t_100m:C,relative_humidity_100m:p,dew_point_100m:C,wind_speed_100m:ms,wind_dir_100m:d
time,,,,,,,,,,,,,,,,,,,,,
2020-01-01 13:00:00,0.0,40.59,26.514689,0.28969,0.0,0.0,1.0,10.0,64826.0,7.4,...,8.4,60.7,1.3,8.4,246.3,8.3,60.3,1.0,10.4,247.3
2020-01-01 14:00:00,0.0,67.86,28.326960,0.29561,0.0,0.0,1.0,0.0,8961.1,0.0,...,8.4,61.6,1.5,8.0,252.3,8.4,60.7,1.2,10.0,252.1
2020-01-01 15:00:00,0.0,116.68,23.682207,0.30044,0.0,0.0,1.0,0.0,0.0,0.0,...,8.5,60.3,1.3,9.6,254.1,8.4,59.6,1.0,11.7,253.8
2020-01-01 16:00:00,0.0,120.22,25.354782,0.29975,0.0,0.0,1.0,0.0,0.0,0.0,...,8.4,63.9,2.0,12.1,254.6,8.3,63.4,1.7,14.3,254.2
2020-01-01 17:00:00,0.0,109.86,23.861942,0.29650,0.0,0.0,1.0,0.0,0.0,0.0,...,7.2,78.9,3.8,11.7,249.5,7.1,77.9,3.5,13.9,249.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31 19:00:00,0.0,21.98,44.422658,0.60243,0.0,0.0,1.0,0.0,0.0,0.0,...,-3.2,93.1,-4.2,4.8,231.2,-3.1,91.6,-4.3,6.0,238.3
2021-01-31 20:00:00,0.0,9.60,45.167707,0.53335,0.0,0.0,1.0,0.0,0.0,0.0,...,-3.5,94.1,-4.3,4.9,224.9,-3.3,92.1,-4.4,6.2,231.8
2021-01-31 21:00:00,0.0,22.61,32.476198,0.51195,0.0,0.0,1.0,0.0,0.0,0.0,...,-3.6,93.0,-4.6,4.7,224.5,-3.3,90.6,-4.6,6.0,231.7


In [5]:
#------------------------------------------- Defining components:
load = Load(name='load',
            time_stamps=data.index,
            p_actual_consumption=data['consumption'],
            critical_load=30,
            max_consumption_increase_rate=50,
            max_consumption_decrease_rate=50,
            installed_capacity=80,
            sink_cost=100,
            source_cost=100,
            growing_price=0.8,
            shedding_price=0.9)

wind_turbine = Renewable(name='wind_turbine',
                         p_actual_generation=data['wind_production'],
                         p_nom=225,
                         sink_cost=80,
                         source_cost=20,
                         curtailment_price=0.3)

pv_plant = Renewable(name='pv_plant',
                     p_actual_generation=data['pv_production'],
                     p_nom=86.4,
                     sink_cost=70,
                     source_cost=10,
                     curtailment_price=0.15)

bess = Storage(name='Battery',
               capacity=500,
               max_charge_rate=400,
               max_discharge_rate=400,
               charging_efficiency=0.98,
               discharging_efficiency=0.98,
               initial_soc=100,
               step_numbers=len(data['consumption']),
               sink_cost=20,
               source_cost=30,
               energy_loss_price=0.075)

hess = Storage(name='Hydrogen',
               capacity=1670,
               max_charge_rate=55,
               max_discharge_rate=100,
               charging_efficiency=0.641,
               discharging_efficiency=0.5,
               initial_soc=334,
               step_numbers=len(data['consumption']),
               sink_cost=30,
               source_cost=40,
               energy_loss_price=0.1)

diesel_generator = Generator(name='Diesel',
                             p_nom=60,
                             length=len(data['consumption']),
                             sink_cost=10,
                             source_cost=90,
                             generation_price=0.5)

exgrid = ExGrid(name='ExGrid',
                p_max_import=100,
                p_max_export=100,
                length=len(data['consumption']),
                sink_cost=60,
                source_cost=60,
                import_price=0.12,
                export_price=0.06)

#---------------------------------------------
microgrid_components = [load, wind_turbine, pv_plant, bess, hess, diesel_generator, exgrid]

#------------------------------------------- Defining Micro-grid:
rye = MicroGrid(components=microgrid_components,
                name='Rye')

In [ ]:
# Defining paths of consumption files:

# poisoned file path:
consumption_file_path = "path/to/your/files/consumption/historical_predictions.json"

# clean file path:
# consumption_file_path = "path/to/your/files/consumption/historical_predictions.json"

# Reading the files:
with open(consumption_file_path, 'r') as f:
    consumption_prediction = json.load(f)

In [ ]:
# Defining paths of wind_production files:

# poisoned file path:
wind_production_file_path = "path/to/your/files/wind_production/historical_predictions.json"

# clean file path:
# wind_production_file_path = "path/to/your/files/wind_production/historical_predictions.json"

# Reading the files:
with open(wind_production_file_path, 'r') as f:
    wind_production_prediction = json.load(f)

In [ ]:
# Defining paths of pv_production files:

# poisoned file path:
pv_production_file_path = "path/to/your/files/pv_production/historical_predictions.json"

# clean file path:
# pv_production_file_path = "path/to/your/files/pv_production/historical_predictions.json"

# Reading the files:
with open(pv_production_file_path, 'r') as f:
    pv_production_prediction = json.load(f)

In [ ]:
def convert_keys_to_int(predictions):
    """
    Converts the string keys of each dictionary to integer (if possible).

    Args:
        predictions (list of dict): A list of dictionaries.

    Returns:
        list of dict: A list of dictionaries with keys converted to integers where possible.
    """
    updated_predictions = []
    for prediction_dict in predictions:
        new_dict = {}
        for key, value in prediction_dict.items():
            try:
                # Attempt to convert the key to an integer
                new_key = int(key)
                new_value = round_value(value)
            except ValueError:
                # If conversion is not possible, the key remains unchanged
                new_key = key
                new_value = round_value(value)
            new_dict[new_key] = new_value
        updated_predictions.append(new_dict)
    return updated_predictions


In [10]:
# Converting keys to int for all three time-series:
consumption_prediction = convert_keys_to_int(consumption_prediction)
wind_prediction = convert_keys_to_int(wind_production_prediction)
pv_prediction = convert_keys_to_int(pv_production_prediction)


In [6]:
# # Using Real values for time-series if needed:
consumption_prediction = create_time_window_dicts_modified(data['consumption'], window_size=forecast_horizon)
wind_prediction = create_time_window_dicts_modified(data['wind_production'], window_size=forecast_horizon)
pv_prediction = create_time_window_dicts_modified(data['pv_production'], window_size=forecast_horizon)

d:\cursor\DP_SIEMS\src\Optimization_function_rounded.py:342: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  window_dict = {hour: round_value(time_series[i + hour]) for hour in range(window_size)}
d:\cursor\DP_SIEMS\src\Optimization_function_rounded.py:350: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  window_dict[hour] = round_value(time_series[i + hour])


## RBC first steps:

In [7]:

rye.run_simulation(num_steps=sequence_length)
res = rye.results()
res


,Time_stamp,load_Actual_Consumption,load_Change,load_Balance,load_Cost,load_Cumulative_Cost,Loads_Cost,Cumulative_Loads_Cost,wind_turbine_Actual_Generation,wind_turbine_Curtailment,...,ExGrid_Exchange,ExGrid_Balance,ExGrid_Cost,ExGrid_Cumulative_Cost,ExGrids_Cost,Cumulative_ExGrids_Cost,Energy_Imbalance,Balancing_delta,Total_Cost,Cumulative_Total_Cost
0,2020-01-01 13:00:00,26.51,0,0,-0.0,-0.0,0.0,0.0,40.59,0,...,0,0,0.0,0.0,0.0,0.0,0.000000e+00,0,0.021120,0.021120
1,2020-01-01 14:00:00,28.33,0,0,-0.0,-0.0,0.0,0.0,67.86,0,...,0,0,0.0,0.0,0.0,0.0,0.000000e+00,0,0.059295,0.080415
2,2020-01-01 15:00:00,23.68,0,0,-0.0,-0.0,0.0,0.0,116.68,0,...,0,0,0.0,0.0,0.0,0.0,7.105427e-15,0,0.139500,0.219915
3,2020-01-01 16:00:00,25.35,0,0,-0.0,-0.0,0.0,0.0,120.22,0,...,0,0,0.0,0.0,0.0,0.0,-7.105427e-15,0,0.142305,0.362220
4,2020-01-01 17:00:00,23.86,0,0,-0.0,-0.0,0.0,0.0,109.86,0,...,0,0,0.0,0.0,0.0,0.0,0.000000e+00,0,0.129000,0.491220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9510,2021-01-31 19:00:00,44.42,0,0,-0.0,-0.0,0.0,0.0,21.98,0,...,0,0,0.0,0.0,0.0,0.0,-2.244000e+01,0,0.000000,0.792718
9511,2021-01-31 20:00:00,45.17,0,0,-0.0,-0.0,0.0,0.0,9.60,0,...,0,0,0.0,0.0,0.0,0.0,-3.557000e+01,0,0.000000,0.792718
9512,2021-01-31 21:00:00,32.48,0,0,-0.0,-0.0,0.0,0.0,22.61,0,...,0,0,0.0,0.0,0.0,0.0,-9.870000e+00,0,0.000000,0.792718
9513,2021-01-31 22:00:00,28.56,0,0,-0.0,-0.0,0.0,0.0,21.70,0,...,0,0,0.0,0.0,0.0,0.0,-6.860000e+00,0,0.000000,0.792718


## MPC for remained steps:

In [8]:
# Whole process of Energy Management System: Using CPLEX
now = datetime.datetime.now()
timestamp_str = now.strftime("%Y%m%d_%H%M%S")
# Getting initial values for battery and hydrogen SoC:
battery_init_SoC = rye.components['storages'][0].soc[sequence_length]
hydrogen_init_SoC = rye.components['storages'][1].soc[sequence_length]

objective_function_prices = {
    0:rye.components['loads'][0].growing_price,
    1:rye.components['loads'][0].shedding_price,
    2:rye.components['generators'][0].generation_price,
    3:rye.components['renewables'][0].curtailment_price,
    4:rye.components['renewables'][1].curtailment_price,
    5:rye.components['storages'][0].energy_loss_price,
    6:rye.components['storages'][1].energy_loss_price,
    7:rye.components['exgrids'][0].import_price,
    8:rye.components['exgrids'][0].export_price}

#----- Setup the solver
solver = pyo.SolverFactory('cplex_direct')

length = len(wind_prediction)

for t in tqdm(range(sequence_length, 9514)):
    
    opti_model = milp_opti_model_connected(objective_function_prices,
                                #  consumption_prediction[t-sequence_length],
                                 consumption_prediction[t],
                                #  wind_prediction[t-sequence_length],
                                 wind_prediction[t],
                                #  pv_prediction[t-sequence_length],
                                 pv_prediction[t],
                                 battery_init_SoC,
                                 hydrogen_init_SoC)
    

    #----- Run the optimization for that specific window: 
    results = solver.solve(opti_model, keepfiles=False,)# logfile="solve.log")
    
    # Extracting forecasted values for current time-step:
    # rye.pv.p_forecasted[t] = pyo.value(opti_model.p_pv[0])
    # rye.wind.p_forecasted[t] = pyo.value(opti_model.p_wind[0])
    # rye.load.p_forecasted[t] = pyo.value(opti_model.p_consumption[0])
    
    # first values of optimized variables are control_commands for each component :
    # For load:
    rye.components['loads'][0].p_change[t] = (pyo.value(opti_model.p_load_grow[0]) -
                                              pyo.value(opti_model.p_load_shed[0]))
    # For diesel generator:
    rye.components['generators'][0].p_gen[t] = pyo.value(opti_model.p_diesel[0])
    # For wind turbine:
    rye.components['renewables'][0].p_curtailment[t] = pyo.value(opti_model.p_wind_curtailment[0])
    # For pv plant:
    rye.components['renewables'][1].p_curtailment[t] = pyo.value(opti_model.p_pv_curtailment[0])
    # For BESS:
    rye.components['storages'][0].p_conv[t] = (pyo.value(opti_model.p_battery_charge[0]) -
                                               pyo.value(opti_model.p_battery_discharge[0]))
    # For HESS:
    rye.components['storages'][1].p_conv[t] = (pyo.value(opti_model.p_hydrogen_charge[0]) -
                                               pyo.value(opti_model.p_hydrogen_discharge[0]))
    # For exgrid:
    rye.components['exgrids'][0].p_exchange[t] = (pyo.value(opti_model.p_exgrid_import[0]) -
                                                  pyo.value(opti_model.p_exgrid_export[0]))
    #----------------
    # Calculating surplus or deficit energy in current step and balance the control_commands:
    rye.balancing(t)
    # for storage in rye.components['storages']:
    #             storage.soc_update(t)
    # updating soc of battery and hydrogen:
    battery_init_SoC = rye.components['storages'][0].soc[t+1]
    hydrogen_init_SoC = rye.components['storages'][1].soc[t+1]
    
    # Deleting optimodel for RAM issues:
    del opti_model
print(f"Generated Timestamp for this run: {timestamp_str}")


100%|██████████| 9490/9490 [06:22<00:00, 24.80it/s]

Generated Timestamp for this run: 20250608_103307


In [9]:
res = rye.results()
res

,Time_stamp,load_Actual_Consumption,load_Change,load_Balance,load_Cost,load_Cumulative_Cost,Loads_Cost,Cumulative_Loads_Cost,wind_turbine_Actual_Generation,wind_turbine_Curtailment,...,ExGrid_Exchange,ExGrid_Balance,ExGrid_Cost,ExGrid_Cumulative_Cost,ExGrids_Cost,Cumulative_ExGrids_Cost,Energy_Imbalance,Balancing_delta,Total_Cost,Cumulative_Total_Cost
0,2020-01-01 13:00:00,26.51,0.0,0,-0.0,-0.0,0.0,0.0,40.59,0.0,...,0.00,0.0,0.0000,0.000000,0.0000,0.000000,0.000000e+00,0.0,0.021120,0.021120
1,2020-01-01 14:00:00,28.33,0.0,0,-0.0,-0.0,0.0,0.0,67.86,0.0,...,0.00,0.0,0.0000,0.000000,0.0000,0.000000,0.000000e+00,0.0,0.059295,0.080415
2,2020-01-01 15:00:00,23.68,0.0,0,-0.0,-0.0,0.0,0.0,116.68,0.0,...,0.00,0.0,0.0000,0.000000,0.0000,0.000000,7.105427e-15,0.0,0.139500,0.219915
3,2020-01-01 16:00:00,25.35,0.0,0,-0.0,-0.0,0.0,0.0,120.22,0.0,...,0.00,0.0,0.0000,0.000000,0.0000,0.000000,-7.105427e-15,0.0,0.142305,0.362220
4,2020-01-01 17:00:00,23.86,0.0,0,-0.0,-0.0,0.0,0.0,109.86,0.0,...,0.00,0.0,0.0000,0.000000,0.0000,0.000000,0.000000e+00,0.0,0.129000,0.491220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9510,2021-01-31 19:00:00,44.42,0.0,0,-0.0,-0.0,0.0,0.0,21.98,0.0,...,22.44,0.0,2.6928,-7840.483389,2.6928,-7840.483389,0.000000e+00,0.0,2.692800,-7597.110064
9511,2021-01-31 20:00:00,45.17,0.0,0,-0.0,-0.0,0.0,0.0,9.60,0.0,...,35.57,0.0,4.2684,-7836.214989,4.2684,-7836.214989,0.000000e+00,0.0,4.268400,-7592.841664
9512,2021-01-31 21:00:00,32.48,0.0,0,-0.0,-0.0,0.0,0.0,22.61,0.0,...,9.87,0.0,1.1844,-7835.030589,1.1844,-7835.030589,0.000000e+00,0.0,1.184400,-7591.657264
9513,2021-01-31 22:00:00,28.56,0.0,0,-0.0,-0.0,0.0,0.0,21.70,0.0,...,6.86,0.0,0.8232,-7834.207389,0.8232,-7834.207389,0.000000e+00,0.0,0.823200,-7590.834064


In [ ]:
# This cell is for naming result files and saving them in a proper way

# building save path:
save_path = "results/"

os.makedirs(save_path, exist_ok=True)

# file naming:

prediction_model = "pure_PyTorch_seq2seq_LSTM"  # or "ARIMA", "RealValues", etc.


# file_name = f"all_three_Poisoned_{prediction_model}_seq_length{sequence_length}_horizon{forecast_horizon}_FGSM_epsilon_{epsilon}_ratio_{ratio}_{timestamp_str}.xlsx"
# file_name = f"all_three_Clean_{prediction_model}_seq_length{sequence_length}_horizon{forecast_horizon}_{timestamp_str}.xlsx"
file_name = f"all_three_Real_{prediction_model}_seq_length{sequence_length}_horizon{forecast_horizon}_{timestamp_str}.xlsx"


# Save the file to the 'results' folder
file_path = os.path.join(save_path, file_name)
res.to_excel(file_path)